# Balances

<div class="alert alert-danger">
  <strong>Warning!</strong> This notebook contains a fake portfolio <i>(automatically generated)</i> and <b>does not</b> represent my own, it merely serves as an example.
</div>


In [ ]:
import pytz
import socket
from datetime import datetime, timezone

now = datetime.now().astimezone(pytz.timezone("America/Los_Angeles"))
print(f"Last executed on {now} on {socket.gethostname()}")

<details>
<summary>Click here for instructions</summary>

### Binance

Uses API and gets both wallet and Binance Earn investments.

### BlockFi

Uses `csv` files downloaded from https://app.blockfi.com/settings/reports

### Nexo.io

Uses `csv` files downloaded from https://platform.nexo.io/transactions

### Exodus

Uses `csv` files which can be extracted using:

Exodus Desktop application → *Exodus* in menu bar → *Developer* → *Export All Transactions*


TODO: get old Exodus wallet

### Trust Wallet

Uses an BCSScan.com API query for all BEP20 tokens.

</details>

In [ ]:
import net_worth_tracker as nwt
balances = {}

In [ ]:
# Crypto
with nwt.utils.try_and_print_error("binance"):
    balances["binance"] = nwt.binance.get_binance_balances()

with nwt.utils.try_and_print_error("blockfi"):
    balances["blockfi"] = nwt.blockfi.get_blockfi_balances()

with nwt.utils.try_and_print_error("nexo"):
    balances["nexo"] = nwt.nexo.get_nexo_balances()

with nwt.utils.try_and_print_error("celsius"):
    balances["celsius"] = nwt.celsius.get_celsius_balances()

with nwt.utils.try_and_print_error("exodus"):
    balances["exodus"] = nwt.exodus.get_exodus()
    balances["exodus"]["SOL"]["amount"] = 5.759

with nwt.utils.try_and_print_error("debank"):
    balances.update(nwt.debank.get_debank_balances())

# Stocks
with nwt.utils.try_and_print_error("degiro"):
    balances["degiro"] = nwt.degiro.get_degiro_balances()

with nwt.utils.try_and_print_error("brand_new_day"):
    brand_new_day = nwt.brand_new_day.scrape_brand_new_day()
    balances["brand_new_day"] = nwt.brand_new_day.get_bnd_balances(brand_new_day)

with nwt.utils.try_and_print_error("manual"):
    manual = nwt.manual.load("manual.yaml")
    balances.update(manual)

# Add crypto prices from CoinGecko

In [ ]:
with nwt.utils.hide("Click here to see nwt.coin_gecko.add_value_and_price output."):
    nwt.coin_gecko.add_value_and_price(balances)
nwt.utils.save_data(balances, None)

## Historic overview

ATH is a personal ATH and 24h and 1w price change accuracy depends on sampling frequency of this notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

import net_worth_tracker as nwt

plt.style.use("dark_background")

pd.options.mode.chained_assignment = None

datas = nwt.utils.load_data(ndays=999)
df = nwt.utils.datas_to_df(
    datas,
    ignore=(
        "degiro",
        "brand_new_day",
        "stock_manual",
    ),
    ignore_symbols=nwt.ignore_symbols,
)

overview = nwt.utils.overview_df(df)
total_ath_value = overview["ATH value (€)"].sum(skipna=True)
total_atl_value = overview["ATL value (€)"].sum(skipna=True)
total_value = overview["value"].sum(skipna=True)
total_stable = sum(
    x.value
    for symbol, x in overview.iterrows()
    if symbol in {"DAI", "miMATIC", "USDC", "USDT", "MIM"}
)
total_debt = sum(x.value for symbol, x in overview.iterrows() if x.value < 0)
with open("current_value", "w") as f:
    f.write(f"{total_value:.2f}")
print(f"Current networth in crypto:\n€{total_value:.2f}\n")
print(f"total debt: €{total_debt:.2f}, total in stables: €{total_stable:.2f}\n")
print(f"The total ATL/ATH value is\n€{total_atl_value:.2f}/€{total_ath_value:.2f}\n")

In [ ]:
pd.set_option("display.float_format", lambda x: "%.5f" % x)
nwt.utils.styled_overview_df(df)

## Denominated in BTC

In [ ]:
df2 = nwt.utils.denominate_in(df, "BTC", norm=1e8)
nwt.utils.styled_overview_df(df2, min_value=100e-8, currency_symbol="丰")

## Denominated in ETH

In [ ]:
df2 = nwt.utils.denominate_in(df, "ETH", norm=1)
nwt.utils.styled_overview_df(df2, min_value=0.001, currency_symbol="Ξ")

# Plots

In [ ]:
%%capture

fig, ax = plt.subplots(figsize=(15, 15))
fig.subplots_adjust(left=0.2, bottom=0.3)
dates = nwt.utils.unique_dt_per_day(df)[::7]

def update(date):
    nwt.plots.plot_pie_at_date(df, date, min_euro=1, fig=fig, ax=ax, show=False)


ani = FuncAnimation(fig, update, frames=dates, repeat=False)
# ani.save("portfolio.mp4")

In [ ]:
from IPython.display import HTML

display(HTML(ani.to_jshtml()))

In [ ]:
nwt.plots.plot_pie_at_date(df, df.date.max())
plt.show()

In [ ]:
cols = [col for col in df.columns if col.startswith("value_in")]
value_in = (
    df[df.date == df.date.max()][cols].sum(skipna=True).sort_values(ascending=False)
)
value_in.index = [col.replace("value_in_", "") for col in value_in.index]
value_in = value_in[value_in > 0.1]  # ignore small values

fig, ax = plt.subplots(figsize=(8, 8))
patches, texts, _ = ax.pie(value_in, labels=value_in.index, autopct="%1.1f%%")
factor = 100 / value_in.sum()
legend_labels = [
    f"{coin} - {factor*amount:1.2f}% - €{amount:.2f}"
    for coin, amount in value_in.items()
]

ax.axis("equal")
plt.tight_layout()
plt.legend(
    patches,
    legend_labels,
    loc="upper left",
    bbox_to_anchor=(-0.25, 1.0),
    fontsize=16,
)
plt.show()

In [ ]:
ov = overview[overview.value > 50]

profits = (ov.price / ov.avg_price - 1).sort_values()
losses = profits[profits < 0]
colors = ["r" if p < 0 else "g" for p in profits]
profits.plot.bar(figsize=(8, 8), ylabel='price/avg_price-1', color=colors)

In [ ]:
((ov.price / ov.avg_price)).sort_values().plot.bar(figsize=(8, 8), ylabel="price / avg_price")

In [ ]:
ov2 = ov[ov.index != "EUR"]
profits = (ov2.amount * (ov2.price - ov2.avg_price)).sort_values()
losses = profits[profits < 0]
colors = ["r" if p < 0 else "g" for p in profits]
fig, ax = plt.subplots()
profits.abs().plot.bar(
    ax=ax,
    figsize=(8, 8),
    logy=True,
    ylabel="profit (€)",
    color=colors,
)

In [ ]:
profits.abs().plot.bar(
    figsize=(8, 8),
    ylabel="profit (€)",
    color=colors,
)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("dark_background")

fig, ax = plt.subplots(figsize=(8, 8))
nwt.plots.plot_barh_at_date_with_profits(
    df, df.date.iloc[-1], min_euro=10, fig=fig, ax=ax, show=True
)

In [ ]:
%%capture

fig, ax = plt.subplots(figsize=(8, 8))
dates = nwt.utils.unique_dt_per_day(df)[::7]

def update(date):
    nwt.plots.plot_barh_at_date(df, date, min_euro=10, fig=fig, ax=ax, show=False)

ani = FuncAnimation(fig, update, frames=dates, repeat=False)

# ani.save("barh.mp4")

In [ ]:
display(HTML(ani.to_jshtml()))

In [ ]:
cols = [col for col in df.columns if col.startswith("value_in")]
value_ratios = df.groupby("date").sum()[cols]
value_ratios.columns = [col.replace("value_in_", "") for col in value_ratios.columns]
value_ratios.plot(
    xlabel="Date", ylabel="Amount (€)", ylim=(0, None), title="Value per category"
)
plt.show()

## Total value over time

In [ ]:
def get_title(series):
    coin = series.name
    pct_change = 100 * (series.iloc[-1] - series.iloc[0]) / series.iloc[0]
    amount_change = series.iloc[-1] - series.iloc[0]
    return (
        f"{coin} value over time, {pct_change:.1f}%, "
        f"€{amount_change:.2f} change since the start"
    )


total = df.groupby("date").sum()["value"]
total.name = "total"
total.plot(
    title=get_title(total),
    xlabel="Date",
    ylabel="Value (€)",
)
plt.show()

## Value over time for current top 15 holdings

In [ ]:
total = df.groupby(["date", "symbol"]).sum()[["value", "amount"]]
total = total.reset_index().set_index("date")
last = total[total.index == total.index.max()]
top_n = 15
biggest_holdings = last.sort_values("value", ascending=False).iloc[:top_n]

for coin in biggest_holdings.symbol:
    sel = total[total.symbol == coin]
    bal = sel.amount
    eur = sel.value
    eur.name = coin
    fig0, ax0 = plt.subplots()
    ax1 = ax0.twinx()
    ax1.spines["right"].set_edgecolor("C1")
    ax1.tick_params(axis="y", colors="C1")
    ax1.set_ylabel(f"Amount in {coin}")
    mid, min_, max_ = bal.mean(), bal.min(), bal.max()
    ax1.set_ylim(min(mid * 0.9, min_), max(mid * 1.1, max_))
    eur.plot(ax=ax0, xlabel="Date", ylabel="Value (€)", title=get_title(eur))
    bal.plot(ax=ax1, c="C1")
    plt.show()